# 목차
> ## 라이브러리 불러오기
> ## 데이터 확인 및 전처리
> ## 데이터 분석
> ## csr matrix 만들기
> ## 모델 학습
> ## 영화 추천받기
> ## 회고

### 필요한 라이브러리를 불러옵니다.

In [1]:
import os
import pandas as pd
from implicit.als import AlternatingLeastSquares
import numpy as np
from scipy.sparse import csr_matrix

### 데이터 확인 및 전처리

* 사람들이 별점을 준 데이터를 불러오겠습니다.

In [2]:
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


* 2점 이하는 추천하기 좀 그러니 3점 이상만 남기겠습니다.

In [3]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


* 3점 이상이 약 84%정도라 삭제해도 문제 없어 보입니다.

* raings 컬럼의 이름을 바꾸고 진행하겠습니다.

In [4]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [60]:
ratings['counts'].head(10)

0    5
1    3
2    3
3    4
4    5
5    3
6    5
7    5
8    4
9    4
Name: counts, dtype: int64

* 1번 유저가 준 영화별 별점 일부를 보겠습니다.

In [6]:
condition = (ratings['user_id']== ratings.loc[0, 'user_id'])
ratings.loc[condition].head()

,user_id,movie_id,counts,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


* movie_id만 가지고는 어떤 영화인지 알 수 없어 메타 데이터를 불러오겠습니다.

In [7]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [8]:
movies.tail()

,movie_id,title,genre
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama
3882,3952,"Contender, The (2000)",Drama|Thriller


* movie_id중 중간에 비는 값이 있는것 같습니다. 주의해야겠네요

* 이제 기존의 ratings 에 타이틀을 합치겠습니다.

In [9]:
idx_to_title, title_to_idx = {}, {}
for i, data in movies[['movie_id', 'title']].iterrows():
    idx_to_title[data.movie_id] = data.title
    title_to_idx[data.title] = data.movie_id

In [76]:
ratings = pd.merge(ratings, movies[['movie_id', 'title']], how='left', on=['movie_id'])
ratings.head()

,user_id,movie_id,counts,title_x,title_y
0,0,1193,5,One Flew Over the Cuckoo's Nest (1975),One Flew Over the Cuckoo's Nest (1975)
1,0,661,3,James and the Giant Peach (1996),James and the Giant Peach (1996)
2,0,914,3,My Fair Lady (1964),My Fair Lady (1964)
3,0,3408,4,Erin Brockovich (2000),Erin Brockovich (2000)
4,0,2355,5,"Bug's Life, A (1998)","Bug's Life, A (1998)"


* merge를 사용해 movie_id기준으로 합쳤습니다.

* timestamp는 불필요하다 생각해 제거하겠습니다.

In [11]:
del ratings['timestamp']

* 확인해 보겠습니다.

### 데이터 분석

In [12]:
ratings.head()

,user_id,movie_id,counts,title
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975)
1,1,661,3,James and the Giant Peach (1996)
2,1,914,3,My Fair Lady (1964)
3,1,3408,4,Erin Brockovich (2000)
4,1,2355,5,"Bug's Life, A (1998)"


* 몇 명의 유저가 있나 확인해보겠습니다.

In [13]:
# 유저 수
ratings['user_id'].nunique()

6039

* 유저들이 본 영화의 수를 보겠습니다.

In [90]:
# 영화 수
ratings['movie_id'].nunique()

3628

* 인기가 많은 영화를 30개까지 확인해 보겠습니다.

In [15]:
# 인기 많은 영화
movie_count = ratings.groupby('title')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

* 익숙한 영화들이 많이 보이네요!

In [16]:
# 유저별 몇 개의 영화를  보았는지에 대한 통계
user_count = ratings.groupby('user_id')['title'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: title, dtype: float64

* 평균적으로 한 유저당 138개의 영화를 보았네요.
* 가장 적게본 사람은 1개만 보았습니다.
* 가장 많이 본 사람은 1968개나 보았네요.

### 제가 선호하는 영화를 5가지 골라서 ratings에 추가해 보겠습니다.

In [17]:
print(movies.loc[movies['title']=='Saving Private Ryan (1998)'])
print(movies.loc[movies['title']=='Toy Story (1995)'])
print(movies.loc[movies['title']=='Terminator, The (1984)'])
print(movies.loc[movies['title']=='Men in Black (1997)'])
print(movies.loc[movies['title']=='Matrix, The (1999)'])

      movie_id                       title             genre
1959      2028  Saving Private Ryan (1998)  Action|Drama|War
   movie_id             title                        genre
0         1  Toy Story (1995)  Animation|Children's|Comedy
      movie_id                   title                   genre
1220      1240  Terminator, The (1984)  Action|Sci-Fi|Thriller
      movie_id                title                           genre
1539      1580  Men in Black (1997)  Action|Adventure|Comedy|Sci-Fi
      movie_id               title                   genre
2502      2571  Matrix, The (1999)  Action|Sci-Fi|Thriller


* title만 추가 할수 없어 각 title별 movie_id를 확인해 보았습니다.

In [18]:
my_favorite = ['Saving Private Ryan (1998)' , 'Toy Story (1995)' ,'Terminator, The (1984)' ,'Men in Black (1997)' ,'Matrix, The (1999)']
my_movie_id = [2028, 1, 1240, 1580, 2571]

# 'ost'이라는 user_id가 위 영화를 별점 5개씩 주었다고 가정하겠습니다.
my_movielist = pd.DataFrame({'user_id': ['ost']*5, 'movie_id' : my_movie_id, 'counts':[5]*5, 'title' : my_favorite})

* 추가하기위한 데이터프레임을 만들었고 확인해 보겠습니다.

In [19]:
my_movielist

,user_id,movie_id,counts,title
0,ost,2028,5,Saving Private Ryan (1998)
1,ost,1,5,Toy Story (1995)
2,ost,1240,5,"Terminator, The (1984)"
3,ost,1580,5,Men in Black (1997)
4,ost,2571,5,"Matrix, The (1999)"


* 추가하겠습니다.

In [20]:
if not ratings.isin({'user_id':['ost']})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
    ratings = ratings.append(my_movielist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

ratings.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,movie_id,counts,title
836473,6040,1090,3,Platoon (1986)
836474,6040,1094,5,"Crying Game, The (1992)"
836475,6040,562,5,Welcome to the Dollhouse (1995)
836476,6040,1096,4,Sophie's Choice (1982)
836477,6040,1097,4,E.T. the Extra-Terrestrial (1982)
0,ost,2028,5,Saving Private Ryan (1998)
1,ost,1,5,Toy Story (1995)
2,ost,1240,5,"Terminator, The (1984)"
3,ost,1580,5,Men in Black (1997)
4,ost,2571,5,"Matrix, The (1999)"


* 맨 밑에 잘 추가가 되었습니다.

* 위에서 movies의 인덱싱과 movie_id가 중간중간 없는게 있어 직접 딕셔너리를 만들겠습니다.

In [77]:
movie_id_to_idx = movies_idx.set_index('movie_id').T.to_dict('records')
movie_id_to_idx = movie_id_to_idx[0]
movie_id_idx = {}
for k, v in movie_id_to_idx.items():
    movie_id_idx[v] = k

* 유저도 만들겠습니다.

In [24]:
# 고유한 유저를 찾아내는 코드
user_unique = ratings['user_id'].unique()

# 유저 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}

* 위에서 만든 딕셔너리 2개를 사용하여 다시 인덱싱하겠습니다.

In [29]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_id_data = ratings['movie_id'].map(movie_id_idx.get).dropna()
if len(temp_movie_id_data) == len(ratings):
    print('movie_id column indexing OK!!')
    ratings['movie_id'] = temp_movie_id_data
else:
    print('movie_id column indexing Fail!!')

ratings

user_id column indexing Fail!!
movie_id column indexing OK!!


,user_id,movie_id,counts,title
0,0,1193,5,One Flew Over the Cuckoo's Nest (1975)
1,0,661,3,James and the Giant Peach (1996)
2,0,914,3,My Fair Lady (1964)
3,0,3408,4,Erin Brockovich (2000)
4,0,2355,5,"Bug's Life, A (1998)"
...,...,...,...,...
0,6039,2028,5,Saving Private Ryan (1998)
1,6039,1,5,Toy Story (1995)
2,6039,1240,5,"Terminator, The (1984)"
3,6039,1580,5,Men in Black (1997)


* 숫자들이 혹시 str인지 확인해 보겠습니다.

In [30]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 836483 entries, 0 to 4
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   user_id   836483 non-null  int64 
 1   movie_id  836483 non-null  int64 
 2   counts    836483 non-null  int64 
 3   title     836483 non-null  object
dtypes: int64(3), object(1)
memory usage: 31.9+ MB


* 정수로 잘 되어있습니다.

### csr matrix 만들기

In [91]:
csr_data = csr_matrix((ratings.counts, (ratings.user_id, ratings.movie_id)), shape = (6040, 3953))
csr_data

<6040x3953 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Row format>

* 6040은 유저의 숫자 입니다.
* 3953은 유저들이 본 영화 숫자가 아닌 movie_id중 중간에 비어있는 id가 있어 비어있는 값들까지 생각하여 만들었습니다.

### 모델을 구성하여 훈련시켜 보겠습니다.

In [32]:
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [33]:
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [34]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3953x6040 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [35]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

###  내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 보겠습니다.

* 초등학생때 TV에서 아버지와 함께 보았던 터미네이터를 보겠습니다.

In [39]:
ost, Terminator = user_to_idx['ost'], movie_id_idx['Terminator, The (1984)']
ost_vector,Terminator_vector = als_model.user_factors[ost], als_model.item_factors[Terminator]

In [40]:
ost_vector

array([ 0.41156933,  0.22174591,  1.1574076 ,  0.603514  , -0.22429219,
        0.09064042,  0.5181275 , -0.05953077,  0.15345305, -0.00209727,
       -0.15052603, -0.5426774 , -0.25555056,  0.36280972, -0.31574735,
       -1.040051  , -0.38456714,  0.01837142, -0.84018475, -0.05612647,
       -0.03664026,  0.3453114 ,  0.16329357,  0.17366076,  0.7387782 ,
       -0.0561108 ,  0.8141249 ,  0.8003956 ,  1.0223374 ,  0.3292095 ,
       -0.3721217 ,  0.5706023 ,  0.03050753, -0.25409895,  0.5103117 ,
       -0.582886  , -0.5765198 ,  1.0255467 , -0.12290797, -0.82637537,
       -0.04104723,  0.45727518,  0.17196544, -0.17751983,  0.04084856,
       -0.6250691 , -0.8833435 ,  0.89913327,  0.02169862, -0.27392918,
       -0.05954449, -0.27383238,  0.75475657,  0.3265246 ,  0.732304  ,
       -0.5696389 ,  0.14705378, -0.07422987, -0.5654029 , -0.49542224,
        0.21554509, -0.4457545 , -0.0188691 ,  0.29783347,  0.6035035 ,
       -0.01188156,  0.52538556,  0.15482412, -0.3333416 , -0.05

In [41]:
Terminator_vector

array([ 1.43272979e-02, -5.75972768e-03,  2.78700329e-02, -1.80807635e-02,
       -9.61119030e-03, -1.07572330e-02,  2.42245998e-02,  2.47842353e-02,
        2.13944279e-02,  8.09609611e-03, -2.84729479e-03,  1.96420308e-03,
       -2.04192586e-02,  1.36880495e-03, -4.42190655e-03, -2.01947838e-02,
       -5.20891976e-04,  4.85806679e-03, -1.28660919e-02, -6.60057785e-03,
        1.74099896e-02,  3.98808680e-02,  1.19086157e-03,  9.30918846e-03,
        1.89521238e-02, -3.15587549e-03,  2.55200081e-02,  1.80625077e-02,
        1.70119032e-02, -9.78682749e-03, -1.23421429e-02,  2.03277785e-02,
        1.41795818e-02, -6.32210588e-03,  1.06635224e-02,  5.57037489e-03,
       -3.63810686e-03,  2.08351903e-05,  5.55233611e-03, -6.69926545e-03,
        2.00692751e-02,  2.58995071e-02,  5.97955659e-03,  3.14559527e-02,
        1.12478938e-02,  1.39065748e-02, -6.08857395e-03,  2.84225084e-02,
        2.50774814e-04,  1.11283492e-02,  2.92893238e-02,  1.52859865e-02,
        3.54664735e-02,  

* 각각 벡터로 바꿨고 내적해보겠습니다.

In [42]:
np.dot(ost_vector, Terminator_vector)

0.4416513

* 44%가 나왔네요. 그리 좋은 값은 아니라고 생각하지만 결과가 어떻게 나올지 모르니 진행하겠습니다.

* 이제 추천받아 보겠습니다.

In [78]:
favorite_movie = 'Terminator, The (1984)'
movie_id = movie_id_idx[favorite_movie]
similar_movie = als_model.similar_items(artist_id, N=15)
similar_movie

[(1240, 1.0),
 (1200, 0.79087734),
 (1036, 0.69888926),
 (3527, 0.6903512),
 (1214, 0.68073404),
 (589, 0.5785759),
 (2571, 0.57114846),
 (2985, 0.53037673),
 (541, 0.52334154),
 (1196, 0.47666493),
 (2916, 0.4700418),
 (1291, 0.45074093),
 (1198, 0.44474474),
 (260, 0.4248583),
 (3703, 0.4167174)]

* 첫번째 값은 자기 자신이기 때문에 1이 나왔습니다.

In [79]:
[movie_id_to_idx[i[0]] for i in similar_movie]

['Terminator, The (1984)',
 'Aliens (1986)',
 'Die Hard (1988)',
 'Predator (1987)',
 'Alien (1979)',
 'Terminator 2: Judgment Day (1991)',
 'Matrix, The (1999)',
 'Robocop (1987)',
 'Blade Runner (1982)',
 'Star Wars: Episode V - The Empire Strikes Back (1980)',
 'Total Recall (1990)',
 'Indiana Jones and the Last Crusade (1989)',
 'Raiders of the Lost Ark (1981)',
 'Star Wars: Episode IV - A New Hope (1977)',
 'Mad Max 2 (a.k.a. The Road Warrior) (1981)']

* 터미네이터 2가 가장 근접 할거라 생각 했었는데 학습이 잘 못된건지 사람들의 선호도가 다른건지 잘 모르겠습니다.

* 위 코드를 함수로 바꿔서 다른 영화도 확인해보겠습니다.

In [80]:
def get_similar_movie(movie_name: str):
    movie_id = movie_id_idx[movie_name]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [movie_id_to_idx[i[0]] for i in similar_movie]
    return similar_movie

In [81]:
get_similar_movie('Toy Story (1995)')

['Toy Story (1995)',
 'Toy Story 2 (1999)',
 "Bug's Life, A (1998)",
 'Aladdin (1992)',
 'Groundhog Day (1993)',
 'Babe (1995)',
 'Lion King, The (1994)',
 'Pleasantville (1998)',
 'Beauty and the Beast (1991)',
 "There's Something About Mary (1998)"]

* 토이스토리는 토이스토리2가 나왔네요

In [82]:
get_similar_movie('Indiana Jones and the Last Crusade (1989)')

['Indiana Jones and the Last Crusade (1989)',
 'Raiders of the Lost Ark (1981)',
 'Indiana Jones and the Temple of Doom (1984)',
 'Die Hard (1988)',
 'Batman (1989)',
 'Untouchables, The (1987)',
 'Highlander (1986)',
 'Terminator, The (1984)',
 'Star Wars: Episode V - The Empire Strikes Back (1980)',
 'Romancing the Stone (1984)']

* 인디아나존스도 다른 모험영화가 나왔습니다.

### 내가 좋아하는 영화와 비슷한 영화를 추천받아 보겠습니다.

In [84]:
user = user_to_idx['ost']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(589, 0.71366274),
 (480, 0.4980299),
 (110, 0.46089587),
 (2916, 0.4440792),
 (457, 0.39046472),
 (3114, 0.37348953),
 (1196, 0.3334297),
 (260, 0.32206374),
 (593, 0.295514),
 (1200, 0.29173863),
 (2628, 0.290281),
 (1210, 0.28578362),
 (1265, 0.26437223),
 (527, 0.26208133),
 (1270, 0.25781187),
 (1610, 0.2551947),
 (1214, 0.24974036),
 (780, 0.24650702),
 (1198, 0.24544322),
 (318, 0.2439285)]

* 71%가 나온 영화가 궁금하네요. 타이틀로 보겠습니다.

In [86]:
[movie_id_to_idx[i[0]] for i in movie_recommended]

['Terminator 2: Judgment Day (1991)',
 'Jurassic Park (1993)',
 'Braveheart (1995)',
 'Total Recall (1990)',
 'Fugitive, The (1993)',
 'Toy Story 2 (1999)',
 'Star Wars: Episode V - The Empire Strikes Back (1980)',
 'Star Wars: Episode IV - A New Hope (1977)',
 'Silence of the Lambs, The (1991)',
 'Aliens (1986)',
 'Star Wars: Episode I - The Phantom Menace (1999)',
 'Star Wars: Episode VI - Return of the Jedi (1983)',
 'Groundhog Day (1993)',
 "Schindler's List (1993)",
 'Back to the Future (1985)',
 'Hunt for Red October, The (1990)',
 'Alien (1979)',
 'Independence Day (ID4) (1996)',
 'Raiders of the Lost Ark (1981)',
 'Shawshank Redemption, The (1994)']

* 터미네이터 2가 나왔습니다. 나름 재밌게 본 기억이 있어 학습이 어느정도 괜찮은것 같습니다.

* 5개의 영화중 어떤 영화가 가장 추천비중이 높았다 확인해보겠습니다.

In [87]:
Terminator_2 = movie_id_idx['Terminator 2: Judgment Day (1991)']
explain = als_model.explain(user, csr_data, itemid=Terminator_2)

In [88]:
[(idx_to_artist[i[0]], i[1]) for i in explain[1]]

[('Matrix, The (1999)', 0.24932961176419127),
 ('Men in Black (1997)', 0.2070570079380535),
 ('Terminator, The (1984)', 0.17173093341504353),
 ('Saving Private Ryan (1998)', 0.05527266017006171),
 ('Toy Story (1995)', 0.019730509590951274)]

* 놀랍게도 터미네이터가 아닌 매트릭스가 가장 높은 비중이였습니다.
* 터미네이터 영화는 계속 예상을 빗나가네요,

### 회고

* 이번 익스가 가장 오류가 많은 익스였습니다.<br/>일단 csr매트릭스를 만들때 처음 오류가 생겼는데 이 오류를 수정하다 더 많은 오류가 생겼고 새싹분들과 다른 대전아이펠 분들의 도움을 받아 해결하였습니다.<br/> 처음 매트릭스를 만들때, 매트릭스의 차원이 초과했다는 오류 메시지가 나왔습니다. 그래서 살펴보니 영화의 개수와 유저들이 본 영화의 개수가 달라서 csr 매트릭스를 만들때 모든 영화를 집어 넣어야 하는데 유저들이 본 영화의 수로 계산해서 오류가 생겼다는걸 알게되었습니다.<br/>그래서 다시 개수를 잡아주니 위에서 영화의 인덱싱을 할때, movie_id가 빈 부분이 있었는데 이 부분을 차례로 딕셔너리로 만들다보니 movie_id와 딕셔너리의 인덱싱넘버가 틀려서 오류가 발생했습니다. <br/>그래서 할 수 없이 df 자체를 딕셔너리로 바꿔주는 함수를 찾아 해결했습니다. 
* 오류를 해결하면서 코드의 이해도가 높아졌고, 토론의 중요성을 또 한번 느꼈습니다.
* csr 매트릭스의 사용하는 이유도 알게되어 앞으로 자연어 공부를 하는데 도움이 많이 될 것 같습니다.
